In [ ]:
%%sql

CREATE TABLE analytics.high_efficiency_category AS
SELECT
  card_tpbuz_nm_1,
  card_tpbuz_nm_2,
  SUM(amt) AS total_amt,
  SUM(cnt) AS total_cnt,
  ROUND(SUM(amt)::FLOAT / NULLIF(SUM(cnt), 0), 2) AS avg_amt_per_txn,
  RANK() OVER (ORDER BY ROUND(SUM(amt)::FLOAT / NULLIF(SUM(cnt), 0), 2) DESC) AS efficiency_rank
FROM raw_data.card_transaction
GROUP BY card_tpbuz_nm_1, card_tpbuz_nm_2
ORDER BY avg_amt_per_txn DESC;
